In [1]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
# method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_EXACT
method = METHOD_SCALAR_QUANTIZE

In [3]:
feedback_bits = 256
ncodebooks=64
ncentroids=256
train_sam_num = 3000 # 训练集样本数
qbits = 8

In [4]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= 'data_to_fc_e39_7999.npy'
    featurepath_train= 'feature_e39_7999.npy'
    data_to_fcpath_test = 'data_to_fc_e39_7999.npy'
    featurepath_test = 'feature_e39_7999.npy'
elif host_name == 'jm-System-Product-Name':
    dir_train = '/data/hdr/transformer_data/joined'
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'data_to_fc_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'feature_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'data_to_fc_test.npy'
    featurepath_test = 'feature_test.npy'
    y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dir_train")


weightpath = 'encoder_fcw.npy'
biaspath = 'encoder_fcb.npy'



In [5]:
def _scalar_quantize(A, axis=0, signed=False, nbits=8):
    unsigned_maxval = float(1 << int(nbits)) - 1

    # # TODO rm
    # # return np.zeros((A.shape[0], 1)), np.ones((A.shape[0], 1)), A
    # # offsets = np.zeros((A.shape[0], 1))
    # offsets = A.min(axis=1, keepdims=True)
    # # scales = maxval / np.ones((A.shape[0], 1))
    # scales = maxval / A.max(axis=1, keepdims=True)
    # Aq = (A - offsets) * scales
    # return offsets, scales, Aq

    # maxval = float(1 << int(nbits)) - 1
    mins = A.min(axis=axis, keepdims=True)
    # A_offset = A - offsets
    ranges = (A - mins).max(axis=axis, keepdims=True) + 1e-20
    print("ranges:")
    print(ranges)
    scales = unsigned_maxval / ranges
    # Aq = (A_offset * (maxval / scales)).astype(np.int)
    # Aq = (A_offset * scales).astype(np.int)

    if signed:
        # sign_offset = 1 << (nbits - 1)  # 8 bits -> 128
        # A_offset -= sign_offset
        offsets = mins + (ranges * (128. / 255))
        minval = -(1 << (nbits - 1))
        maxval = -minval - 1
    else:
        offsets = mins
        minval = 0
        maxval = (1 << nbits) - 1

    Aq = (A - offsets) * scales
    # print("min, max A:", Aq.min(), Aq.max())  # looks good
    Aq = np.clip(Aq, minval, maxval).astype(int)

    return offsets, scales, Aq, minval, maxval

In [6]:
def test_quantize(A, B, offsets, scales, minval, maxval, axis=0, signed=False, nbits=8):
    Aq = (A - offsets) * scales
    Aq = np.clip(Aq, minval, maxval).astype(int) # 量化后的值（整数
    Aqreal = Aq / scales + offsets # 量化后对应的实际值
    print("A:")
    print(A)
    print("Aq:")
    print(Aq)
    print("Aqreal")
    print(Aqreal)
    C = np.matmul(Aqreal, B)

    return C

In [7]:
ran_mat = np.rint(10*np.random.random(size=(5,5)))
print(ran_mat)

[[ 9.  6.  3.  5.  2.]
 [ 1.  8.  1.  1.  2.]
 [ 8.  1.  9.  1.  6.]
 [ 5.  0.  2.  1. 10.]
 [ 7.  7.  4.  3.  5.]]


In [8]:
offsets, scales, Aq, minval, maxval = _scalar_quantize(ran_mat, axis=0, signed=False, nbits=8)

ranges:
[[8. 8. 8. 4. 8.]]


In [9]:
print(offsets)
print(scales)
print(Aq)
print(minval)
print(maxval)

[[1. 0. 1. 1. 2.]]
[[31.875 31.875 31.875 63.75  31.875]]
[[255 191  63 255   0]
 [  0 255   0   0   0]
 [223  31 255   0 127]
 [127   0  31   0 255]
 [191 223  95 127  95]]
0
255


In [10]:
data_to_fc_train = np.load(os.path.join(dir_train, data_to_fcpath_train))
print(data_to_fc_train.shape)
weight = np.load(os.path.join(dir_train, weightpath))
print(weight.shape)
data_to_fc_test = np.load(os.path.join(dir_train, data_to_fcpath_test))
print(data_to_fc_test.shape)
bias = np.load(dir_train+'/'+biaspath)

(96000, 2048)
(2048, 256)
(32000, 2048)


In [11]:
train_offsets, train_scales, train_Aq, train_minval, train_maxval = _scalar_quantize(data_to_fc_train, axis=0, signed=False, nbits=qbits)

ranges:
[[0.01557301 0.00956542 0.01400521 ... 0.17773968 0.14837046 0.15120666]]


In [12]:
y_out_matmul = test_quantize(data_to_fc_test, weight, train_offsets, train_scales, train_minval, train_maxval)
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

A:
[[ 0.001161   -0.0003292   0.00079327 ...  0.02352978  0.02011615
   0.01869451]
 [-0.00108941 -0.0003551  -0.00034218 ...  0.01012411  0.00046477
   0.00978941]
 [ 0.00097858  0.00035693  0.00280184 ... -0.01709364  0.01152093
   0.00830318]
 ...
 [ 0.0044448  -0.00065186  0.00043888 ... -0.0145436  -0.00225845
   0.00354694]
 [ 0.00094269 -0.0011471   0.00078288 ...  0.00279342  0.00330969
  -0.01324401]
 [ 0.00324066  0.00071579  0.00377456 ... -0.0036882   0.00070076
  -0.00470257]]
Aq:
[[136 128 136 ... 147 158 169]
 [ 99 127 116 ... 128 124 154]
 [133 146 173 ...  89 143 151]
 ...
 [190 119 130 ...  93 119 143]
 [133 106 136 ... 118 129 115]
 [170 155 191 ... 108 124 129]]
Aqreal
[[ 0.00111608 -0.00033059  0.00074773 ...  0.0228645   0.02008874
   0.01843729]
 [-0.00114353 -0.00036811 -0.00035071 ...  0.00962115  0.00030602
   0.00954278]
 [ 0.00093287  0.00034461  0.00277986 ... -0.01756257  0.01136107
   0.00776388]
 ...
 [ 0.0044139  -0.0006682   0.0004182  ... -0.01477449 

In [13]:
print(y_out_last)
print(y_out_last.dtype)
print(y_out_last.shape)
np.save(dir_result+'/'+'UserQuantize'+'fc1_fb%i_qb%i.npy' % (feedback_bits, qbits), y_out_last.astype(np.float32))


[[-0.08795245  0.01847343 -0.54548272 ... -0.07523059  0.07883646
   0.00561524]
 [-0.23517419 -0.12676349 -0.29348168 ... -0.38049333  0.00978337
  -0.07713263]
 [ 0.3898445  -0.05268517 -0.24185705 ... -0.50888443  0.18536493
   0.11724634]
 ...
 [ 0.3498211   0.14093997 -0.52193499 ... -0.1764249  -0.20703671
   0.00418148]
 [ 0.24449094  0.01195635 -0.24397657 ...  0.05938521 -0.00413316
   0.05226247]
 [-0.31105483  0.10657356 -0.42814031 ... -0.34607515 -0.14545673
   0.17339793]]
float64
(32000, 256)


In [14]:
print(train_offsets.shape)
print(train_scales.shape)
print(train_Aq.shape)

(1, 2048)
(1, 2048)
(96000, 2048)


In [15]:
print(train_offsets)

[[-0.00718952 -0.00513206 -0.00672171 ... -0.0795972  -0.07184276
  -0.08177418]]
